In [66]:
import numpy as np

# Load data

In [67]:
from json import load

with open('agora_hack_products.json', encoding='utf-8') as file:
    all_products = load(file)

all_products

[{'product_id': '0007302f2fe1d54d',
  'name': 'Классическая сплит-система ROYAL CLIMA PANDORA RC-PD28HN, для комнат до 28 кв.метра, настенный кондиционер для дома/офиса, комплект',
  'props': ['Класс  энергоэффективности\tA',
   'Мощность  кондиционера\t9 BTU',
   'Уровень  шума внутреннего блока\t21.5 дБ - 38 дБ',
   'Режим   работы\tохлаждение / обогрев',
   'Фильтр тонкой очистки\tесть',
   'Доп.   режимы турбо, экорежим, осушение, ночной, вентиляция'],
  'is_reference': False,
  'reference_id': 'f497219eb0077f84'},
 {'product_id': '000740b6c1cc763e',
  'name': 'Смартфон Xiaomi Redmi Note 10S NFC 6/128 ГБ RU, белоснежная галька',
  'props': ['Экран\t6.43" (2400x1080) AMOLED 60 Гц',
   '4 камеры 8 МП, 64 МП, 2 МП, 2 МП',
   'Беспроводные интерфейсы инфракрасный порт (IRDA), Bluetooth, Wi-Fi, NFC',
   'Встроенная   память\t128 ГБ',
   'Оперативная   память\t6 ГБ',
   'Аккумулятор\t5000  мА·ч',
   'MediaTek   Helio G95',
   'SIM-карты\t2  (nano SIM)',
   'Операционная  система\tAndroid

In [68]:
class Product:
    def __init__(self, product_id: str, name: str, props: [str], is_reference: bool, reference_id: str):
        self.product_id = product_id
        self.name = name
        self.props = props
        self.is_reference = is_reference
        self.reference_id = reference_id

    def __str__(self):
        return f"{'Reference' if self.is_reference else f'Referrer of {self.reference_id}'} {self.name} ({self.product_id}): \n {self.props}"

In [69]:
all_products = [Product(**p) for p in all_products]

In [70]:
references = [p for p in all_products if p.is_reference]
references_id_set = set([ref.product_id for ref in references])
products = [p for p in all_products if p.product_id not in references_id_set]

print(*references)

Reference Смартфон Xiaomi Redmi Note 10S NFC 6/128 ГБ RU, белоснежная галька (000740b6c1cc763e): 
 ['Экран\t6.43" (2400x1080) AMOLED 60 Гц', '4 камеры 8 МП, 64 МП, 2 МП, 2 МП', 'Беспроводные интерфейсы инфракрасный порт (IRDA), Bluetooth, Wi-Fi, NFC', 'Встроенная   память\t128 ГБ', 'Оперативная   память\t6 ГБ', 'Аккумулятор\t5000  мА·ч', 'MediaTek   Helio G95', 'SIM-карты\t2  (nano SIM)', 'Операционная  система\tAndroid 11', 'Степень  защиты\tIP53', 'Вес\t179  г', 'Стандарт   связи 2G, 4G LTE, 3G'] Reference Фен LUMME LU-1058 (0083737f904dd9a9): 
 ['Мощность\t1400 Вт', 'Насадки\tконцентратор', 'Вес\t350  г', 'Особенности   складная ручка, защита от перегрева, петля для подвешивания'] Reference 15.6" Ноутбук Lenovo IdeaPad 3 15ARE05 1920x1080, AMD Ryzen 5 4500U 2.3 ГГц, SSD 256 ГБ, AMD Radeon Graphics, DOS, 81W40033RK, Platinum Grey (01378fa92f901df5): 
 ['Экран\t15.6"  (1920x1080) IPS', 'Процессор\tAMD   Ryzen 5 4500U (6x2.30 ГГц)', 'Время   работы от аккумулятора\t14 ч', 'Операционная

In [71]:
def get_referrers(ref: Product):
    assert ref.is_reference
    return [p for p in products if p.reference_id == ref.product_id] # TODO: use filter


def describe_reference(ref: Product):
    print(ref)
    print()
    print(*get_referrers(ref), sep='\n')

In [72]:
describe_reference(references[11])

Reference Микроволновая печь LG MS2535GIS, черный (070dd48327cad4d5): 
 ['Объем\t25 л', 'Инверторное управление мощностью\tда', 'Внутреннее покрытие камеры\tэмаль', 'Переключатели\tсенсорные', 'Мощность  микроволн\t1000 Вт', 'Диаметр  поддона\t292 мм', 'Режимы   работы\tразморозка', 'ШхВхГ\t47.60х27.20х36.80   см', 'Особенности   подсветка камеры, дисплей, блокировка от детей', 'Доп.  режимы автоматический разогрев, программирование процесса приготовления, звуковой сигнал отключения, автоматическое поддержание температуры, автоматическое приготовление, автоматическая разморозка']

Referrer of 070dd48327cad4d5 Микроволновая печь LG MS2535GIS черный (инвертор) (41b312152ddb3af7): 
 ['Переключатели\tсенсорные', 'Внутреннее   покрытие камеры\tэмаль', 'Диаметр  поддона\t292 мм', 'Режимы  работы\tразморозка', 'ШхВхГ\t47.60х27.20х36.80  см', 'Особенности  подсветка камеры, дисплей, блокировка от детей', 'Доп.  режимы звуковой сигнал отключения, автоматическая разморозка, автоматическое поддер

In [73]:
from sklearn.model_selection import train_test_split


products_train, products_test = train_test_split(
    products, test_size=0.4, random_state=42
)
all_products = products_train + references

In [74]:
'''
chars = {}


def calc_chars(s):
    for c in s:
        if c in chars:
            chars[c] += 1
        else:
            chars[c] = 1


for product in all_products:
    calc_chars(product.name)
    for prop in product.props:
        calc_chars(prop)

import numpy as np

if ' ' in chars:
    chars.pop(' ')
if '\t' in chars:
    chars.pop('\t')

char_keys = sorted(list(chars.keys()))
char2ind = {c : i for i, c in enumerate(char_keys)}


def build_vector(s):
    v = np.zeros((len(char_keys)), dtype=int)
    for c in s:
        if c in char2ind:
            v[char2ind[c]] += 1
    return v.astype(float)
'''

"\nchars = {}\n\n\ndef calc_chars(s):\n    for c in s:\n        if c in chars:\n            chars[c] += 1\n        else:\n            chars[c] = 1\n\n\nfor product in all_products:\n    calc_chars(product.name)\n    for prop in product.props:\n        calc_chars(prop)\n\nimport numpy as np\n\nif ' ' in chars:\n    chars.pop(' ')\nif '\t' in chars:\n    chars.pop('\t')\n\nchar_keys = sorted(list(chars.keys()))\nchar2ind = {c : i for i, c in enumerate(char_keys)}\n\n\ndef build_vector(s):\n    v = np.zeros((len(char_keys)), dtype=int)\n    for c in s:\n        if c in char2ind:\n            v[char2ind[c]] += 1\n    return v.astype(float)\n"

In [75]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

def get_product_payload(product: Product) -> str:
    return product.name + ' ' + ' '.join(product.props)

def tokenize(payload: str, language='russian') -> [str]:
    words = word_tokenize(payload, language=language)
    words = list(map(lambda w: w.lower(), words))
    words = list(filter(lambda w: w not in stopwords.words(language), words))

    return words

In [76]:
#nltk.download('punkt')
#nltk.download('stopwords')

In [77]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from spacy.lang.ru import STOP_WORDS as RU_STOP_WORDS
from spacy.lang.en import STOP_WORDS as EN_STOP_WORDS
from spacy.lang.ru import Russian

def products2corpus(prods):
    return [p.name + ' ' + ' '.join(p.props) for p in prods]

train_corpus = products2corpus(all_products)
test_corpus = products2corpus(products_test)

tfidf = TfidfVectorizer(  #TfidfVectorizer(sublinear_tf=True, norm='l2', ngram_range=(1, 2),
    stop_words=RU_STOP_WORDS or EN_STOP_WORDS)#,  tokenizer=tokenize)
X_train = tfidf.fit_transform(train_corpus).toarray()

X_test = tfidf.transform(test_corpus).toarray()

In [78]:
tfidf.build_tokenizer()("Переключатели\tсенсорные', 'Внутреннее   покрытие камеры\tэмаль', 'Диаметр  поддона\t292 мм', 'Режимы  работы\tразморозка', 'ШхВхГ\t47.60х27.20х36.80  см', 'Особенности  подсветка камеры, дисплей, блокировка от дете")

['Переключатели',
 'сенсорные',
 'Внутреннее',
 'покрытие',
 'камеры',
 'эмаль',
 'Диаметр',
 'поддона',
 '292',
 'мм',
 'Режимы',
 'работы',
 'разморозка',
 'ШхВхГ',
 '47',
 '60х27',
 '20х36',
 '80',
 'см',
 'Особенности',
 'подсветка',
 'камеры',
 'дисплей',
 'блокировка',
 'от',
 'дете']

In [79]:
ref_id2ind = {
    ref.product_id : i for i, ref in enumerate(references)
}

def build_target(prods):
    return np.array([
        ref_id2ind[p.product_id if p.is_reference else p.reference_id] for p in prods
    ])

y_train = build_target(all_products)

y_test = build_target(products_test)

In [87]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import RidgeClassifier, SGDClassifier
from sklearn.base import ClassifierMixin

models = [
    RandomForestClassifier(random_state=0),
    LinearSVC(),
    MultinomialNB(),
    LogisticRegression(random_state=0),
    RidgeClassifier(),
    SGDClassifier()
]


def test_model(model: ClassifierMixin):
    model_name = model.__class__.__name__
    model.fit(X_train, y_train)

    train_acc = model.score(X_train, y_train)
    test_acc = model.score(X_test, y_test)

    print(model_name, f"train: {train_acc}, test: {test_acc}")


for model in models:
    test_model(model)

RandomForestClassifier train: 0.9985974754558204, test: 0.8453237410071942
LinearSVC train: 0.9981299672744273, test: 0.9442446043165468
MultinomialNB train: 0.5568022440392707, test: 0.2670863309352518
LogisticRegression train: 0.7554932211313699, test: 0.4721223021582734
RidgeClassifier train: 0.9934548854604955, test: 0.9217625899280576
SGDClassifier train: 0.9939223936418887, test: 0.8803956834532374


In [82]:
from sklearn.svm import LinearSVC, SVC
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import SGDClassifier

'''
grid = GridSearchCV(
    SGDClassifier(verbose=1), {
    'loss' : ['hinge', 'log_loss', 'log', 'modified_huber', 'squared_hinge',
     'perceptron', 'squared_error', 'huber', 'epsilon_insensitive',
     'squared_epsilon_insensitive'],
    'penalty' : ['l2', 'l1', 'elasticnet'],
    'alpha' : [1e-3, 1e-4]
}, scoring=lambda model, x, y: model.score(x, y), n_jobs=10, verbose=10, cv=3)
'''

grid = GridSearchCV(LinearSVC(), {
    'penalty' : ['l1', 'l2'],
    'loss' : ['hinge', 'squared_hinge'],
    'C' : np.arange(0, 1, 0.1),

}, scoring=lambda model, x, y: model.score(x, y), n_jobs=10, verbose=1)


grid.fit(
    np.concatenate([X_train, X_test]),
    np.concatenate([y_train, y_test])
)

'''
'penalty' : ['l1', 'l2'],
'loss' : ['hinge', 'squared_hinge'],
'C' : np.arange(0, 1, 0.1),
'''

Fitting 5 folds for each of 40 candidates, totalling 200 fits


C:\Users\densh\.conda\envs\clip\lib\site-packages\sklearn\model_selection\_split.py:680: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(
C:\Users\densh\.conda\envs\clip\lib\site-packages\sklearn\model_selection\_validation.py:378: FitFailedWarning: 
110 fits failed out of a total of 200.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
50 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\densh\.conda\envs\clip\lib\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\densh\.conda\envs\clip\lib\site-packages\sklearn\svm\_classes.py", line 257,

"\n'penalty' : ['l1', 'l2'],\n'loss' : ['hinge', 'squared_hinge'],\n'C' : np.arange(0, 1, 0.1),\n"

In [84]:
grid.best_params_

{'C': 0.9, 'loss': 'squared_hinge', 'penalty': 'l2'}

In [85]:
grid.best_score_

0.9630859033439678

In [ ]:
0/0

## Best models so far

In [13]:
LinearSVC(C=0.9)

In [ ]:
ref_id2ind = {
    ref.product_id : i for i, ref in enumerate(references)
}


def build_dataset(prods):
    x = np.array([
        sum(map(build_vector, [p.name] + p.props)) for p in prods
    ])
    x = x / np.linalg.norm(x, axis=1, keepdims=True)

    y = np.array([
         ref_id2ind[p.product_id if p.is_reference else p.reference_id] for p in prods
    ])

    return x, y

In [ ]:
x_train, y_train = build_dataset(references + products_train)
x_test, y_test = build_dataset(products_test)

In [ ]:
x_train.shape

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn_model = KNeighborsClassifier()

In [ ]:
from sklearn.model_selection import GridSearchCV

grid = {'n_neighbors' : [1,4], 'algorithm' : ['auto', 'ball_tree', 'kd_tree', 'brute']}
grid_search = GridSearchCV(estimator=knn_model, param_grid=grid)
grid_search.fit(x_train, y_train)

In [ ]:
print(grid_search.best_params_)

In [ ]:
grid_search.score(x_test, y_test)

In [ ]:
!python -m spacy download ru_core_news_sm

In [ ]:
import spacy

nlp = spacy.load("ru_core_news_sm")
doc = nlp(products[0].name)
print(doc.text)
for token in doc:
    print(token.text, token.pos_, token.dep_)
